In [6]:
import cv2
import mediapipe as mp
import time
import os

# Папка для сохранения фото
save_dir = "C:/handdetection"
os.makedirs(save_dir, exist_ok=True)

mp_hands = mp.solutions.hands

# Инициализация детектора рук
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.7,
                       min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

photo_taken = False  # Флаг, чтобы не делать много фото подряд
photo_time = 0       # Время, когда было сделано фото

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        if not photo_taken:
            # Сохраняем "чистый" кадр, без выделений
            clean_frame = frame.copy()
            filename = os.path.join(save_dir, f"hand_photo_{int(time.time())}.jpg")
            cv2.imwrite(filename, clean_frame)
            print(f"Фото сделано и сохранено как {filename}")
            photo_taken = True
            photo_time = time.time()
    else:
        photo_taken = False

    # Показываем надпись "Фото сделано!" 2 секунды после снимка
    if photo_taken and (time.time() - photo_time) < 2:
        cv2.putText(frame, "Photo has been taken!", (50, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    # Показываем изображение (можно убрать, если не нужно окно)
    cv2.imshow('Hand Detector', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()